In [32]:
import pandas as pd
import psycopg2
import warnings
import numpy as np
from psycopg2 import extras

warnings.filterwarnings("ignore")

In [57]:
db_params = {
    'dbname': 'hw1',
    'user': 'postgres',
    'password': 'geheim',
    'host': 'localhost'
}
df = pd.read_excel("data/customer_and_transaction__2_.xlsx", None)

In [139]:
def replace_nan(df):
    for col in df.columns[df.isna().any()].tolist():
        df[col] = df[col].replace({np.nan: None})

In [140]:
customer_full_df = df['customer']
replace_nan(customer_full_df)
cust_df = customer_full_df[['customer_id', 'first_name', 'last_name', 'gender', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car']]
addr_df = customer_full_df[['address', 'postcode', 'state', 'country', 'property_valuation', 'customer_id']]
customer_data = tuple(cust_df.itertuples(index=False, name=None))
addr_data = tuple(addr_df.itertuples(index=False, name=None))

In [179]:
insert_customers_q = """
    INSERT INTO customers (
        id, first_name, last_name, gender, DOB, job_title, job_industry_category,
        wealth_segment, deceased_indicator, owns_car
    ) VALUES %s RETURNING id
"""
insert_addresses_q = """
    INSERT INTO addresses (
        address, postcode, state, country, property_valuation, customer_id
    ) VALUES %s
"""
insert_products_q = """
    INSERT INTO products (
        legacy_product_id, brand, line, class, size, list_price, standard_cost
    ) VALUES %s
"""
insert_transactions_q = """
    INSERT INTO transactions (
        id, customer_id, date, online_order, order_status, product_id
    ) VALUES %s
"""

In [98]:
def insert_data(query, data):
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()
    try:
        extras.execute_values(cur, query, data)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        raise
    finally:
        # Close communication with the database
        if conn is not None:
            conn.close()
    
    print('Data inserted successfully')

In [107]:
insert_data(insert_customers_q, customer_data)
insert_data(insert_addresses_q, addr_data)

Data inserted successfully
Data inserted successfully


In [291]:
transaction_full_df = df['transaction']
replace_nan(transaction_full_df)
product_df = transaction_full_df[['product_id', 'brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost']]
product_df["standard_cost"] = product_df["standard_cost"].astype(float)
product_df = product_df.round(2)
replace_nan(product_df)
product_df = product_df.drop_duplicates()
product_data = tuple(product_df.itertuples(index=False, name=None))

product_df.reset_index(inplace=True)
product_df['index'] = product_df.index + 1

transaction_df = transaction_full_df[[
    'transaction_id', 'product_id', 'customer_id', 'transaction_date', 'online_order', 'order_status', 'list_price'
]]
transaction_df = transaction_df.merge(product_df[['product_id', 'list_price', 'index']], on=['product_id', 'list_price'], how='left')
transaction_df.drop(columns=['product_id', 'list_price'], inplace=True)
transaction_df = transaction_df[transaction_df["customer_id"] <= 4000]
transaction_data = tuple(transaction_df.itertuples(index=False, name=None))

In [292]:
insert_data(insert_products_q, product_data)
insert_data(insert_transactions_q, transaction_data)

Data inserted successfully
Data inserted successfully
